<a href="https://colab.research.google.com/github/dinma-e/Loan-Default-Prediction/blob/main/Predicting_Loan_Default_Risk_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
Financial institutions face a constant challenge: balancing the need to provide loans with the risk of borrowers defaulting. Every default not only leads to financial loss but also affects trust and long-term sustainability. With the rise of digital lending platforms, the ability to anticipate potential defaults has become more important than ever.

As part of my data science journey, I worked on this project as a capstone to apply machine learning techniques to the problem of loan default prediction. The goal was not just to see how well models could classify borrowers, but also to uncover meaningful insights from the data. Rather than focusing only on accuracy, I aimed to understand the dataset, build predictive models, evaluate their performance, and reflect on the lessons learned.

# Problem Statement
Develop a robust machine learning pipeline to predict loan default risk, enabling better credit decisions and minimizing financial losses.

# Objectives
The main objectives of this project are:
1. Data Understanding, Analysis & Preparation: Explore the data sets, gain insights and preprocess the data for model development (handle missing values, encode categorical variables, and scale features).
2. Model Development: Build and test machine learning models to classify borrowers as likely defaulters or non defaulters.
3. Evaluation & Insights: Assess performance using metrics such as recall, precision, and F1-score, with a focus on identifying high risk borrowers.
4. Lessons & Recommendations: Reflect on the challenges faced and share insights that could inform future projects or real world applications.

In [ ]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import geopandas as gpd
import shap

from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectKBest, mutual_info_classif, RFE, RFECV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report, make_scorer, recall_score
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE, ADASYN

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [ ]:
# data collection - get the data sets
url1 = 'https://raw.githubusercontent.com/Oyeniran20/axia_cohort_8/refs/heads/main/trainperf.csv'
url2 = 'https://raw.githubusercontent.com/Oyeniran20/axia_cohort_8/refs/heads/main/traindemographics.csv'
url3 = 'https://raw.githubusercontent.com/Oyeniran20/axia_cohort_8/refs/heads/main/trainprevloans.csv'

# convert to data frames
df_demographics = pd.read_csv(url2)
df_perf = pd.read_csv(url1)
df_prevloans = pd.read_csv(url3)

## Data Understanding and Cleaning
- Examine each dataframe to understand its purpose and content
- Perform basic cleaning like removing duplicates, dropping columns with a high percentage of missing values, changing data types.

#### Column meanings
|Column|Meaning|
|------|-------|
|customerid|A unique identifier for each customer|
|systemloanid|A unique identifier for each loan|
|loannumber|A count of how many loans a customer has taken|
|approveddate|The date the loan was approved|
|creationdate|The date the loan application was submitted|
|loanamount|The amount approved and loaned to the customer (principal amount)|
|totaldue|The amount to be paid back by the customer (principal + interest)|
|termdays|The loan duration in days|
|good_bad_flag|Indicates if the customer defaulted or not (Target column)|
|birthdate|The date of birth of the customer|
|bank_account_type|The type of bank account the customer has|
|longitude_gps|The customer's location longitude|
|latitude_gps|The customer's location latitude|
|bank_name_clients|The name of the customer's bank|
|employment_status_clients|The employment status of the customer|
|level_of_education_clients|The highest level of education of the customer|
|closeddate|The date the loan was officially closed|
|firstduedate|The first due date of repayment|
|firstrepaiddate|The date of the first repayment made by the customer|

### Data Understanding

In [ ]:
# performance dataframe
df_perf.head()

In [ ]:
# previous loans dataframe
df_prevloans.head()

In [ ]:
# demographics dataframe
df_demographics.head()

In [ ]:
# check for common points in dataframes - using customerid 8a2a81a74ce8c05d014cfb32a0da1049

In [ ]:
# performance
df_perf[df_perf['customerid'] == '8a2a81a74ce8c05d014cfb32a0da1049']

In [ ]:
# demographics
df_demographics[df_demographics['customerid'] == '8a2a81a74ce8c05d014cfb32a0da1049']

In [ ]:
# previous loans
df_prevloans[df_prevloans['customerid'] == '8a2a81a74ce8c05d014cfb32a0da1049']

In [ ]:
# create a dictionary containing the data frames for examination purposes
dataframes = {
    'df_demographics' : df_demographics,
    'df_perf' : df_perf,
    'df_prevloans' : df_prevloans}

#### Shape

In [ ]:
for name, data in dataframes.items():
    print(f"Shape of {name} = {data.shape}")

#### Duplicates

In [ ]:
# general
for name, data in dataframes.items():
    print(f"Number of duplicates in {name} = {data.duplicated().sum()}")

In [ ]:
# customer id
for name, data in dataframes.items():
    print(f"Number of duplicates in {name} = {data['customerid'].duplicated().sum()}")

#### Missing Values (percentage)

In [ ]:
for name, data in dataframes.items():
    print(f"Percentage of missing values in {name}:")
    print(f"{(data.isna().sum().sort_values(ascending=False)/len(data))*100}")
    print('---' * 20)

#### Data Types

In [ ]:
for name, data in dataframes.items():
    print(f"Data Types of Columns in {name}:")
    print(f"{data.dtypes}")
    print('---' * 20)

#### **Observations:**
- The performance dataframe (df_perf) contains unique customer loan information and has the column 'good_bad_flag' that tells us whether a customer defaulted or not. This 'good_bad_flag' is the target column
- The demographics dataframe is additional information on the customers such as date of birth and employment status
- The previous loans dataframe is historical information on loans given to current customers
- The previous loans dataframe has a many to one relationship on the customer id. There are 13,824 duplicates of the customerid in df_prevloans
- The dataframes have different shapes:
  - Shape of df_demographics = 4346 rows, 9 columns
  - Shape of df_perf = 4368 rows, 10 columns
  - Shape of df_prevloans = 18,183 rows, 12 columns
- There are 12 duplicates overall in df_demographics
- The bank_branch_clients column in demographics is missing approximately 98.8 percent of values
- The level_of_education_clients column in demographics is missing approximately 86.5 percent of values
- The employment_status_clients in demographics is missing approximately 14.9 percent of values
- The referredby column is missing approximately 86.6 percent and 94.4 percent of values in df_perf and df_prevloans respectively
- The date columns are of the wrong data type - object
  
#### **Insights:**
- Some customers are missing demographic information
- The demographics dataframe should be merged to the performance dataframe on the customer id because the performance dataframe contains the target column and the customer id is the common point between them.
- A single customerid in the previous loans dataframe comes with multiple unique sets of information (the number of general duplicates was observed to be 0) and the columns are not fully in accordance with that of the performance dataframe.
- The previous loans dataframe cannot be merged directly due to its duplicates in the customerid.
- Although the previous loans dataframe cannot be merged directly with the others, new features that are unique to a customerid can be engineered from it which can then be merged to the main dataframe (performance + demographics). It can also be used for analysis to gain more insights.

#### **To Do:**
- Change the data type of the date columns to datetime
- Drop the duplicates in df_demographics
- Drop the referredby, bank_branch_clients columns and level_of_education_clients as there are too many missing values to gain any valuable insights from them

### Data Cleaning

In [ ]:
# change date data types
date_cols = ['approveddate', 'creationdate', 'birthdate', 'firstrepaiddate', 'firstduedate', 'closeddate']

for col in date_cols:
    for name, data in dataframes.items():
        if col in data.columns:
            data[col] = pd.to_datetime(data[col])

df_perf = dataframes['df_perf']
df_prevloans = dataframes['df_prevloans']
df_demographics = dataframes['df_demographics']

In [ ]:
# drop duplicates in demographics
df_demographics.drop_duplicates(inplace=True)

In [ ]:
# drop columns with high percentage of missing values
df_demographics = df_demographics.drop(columns=['bank_branch_clients', 'level_of_education_clients'])
df_perf = df_perf.drop(columns='referredby')
df_prevloans = df_prevloans.drop(columns='referredby')

## Feature Engineering and Merging
- Engineer new features from the previous loans dataframe
- Merge dataframes
- Engineer new features from the merged data

### Feature Engineering - Previous Loans

#### For Merging

In [ ]:
# create a function that determines a default/no default on past loans
def loan_default(data):
    # loan duration
    data['loan_duration'] = (data['closeddate'] - data['approveddate']).dt.days

    # term delay
    data['term_delay'] = data['loan_duration'] - data['termdays']

    # no default
    data['no_default'] = np.where((data['term_delay'] <= 0), 1, 0)

    return data

In [ ]:
# create a fuction that engineers unique features on prevloans
def merge_prevloans(data):
    # apply customer default function
    data = loan_default(data)

    # no. of previous defaults/no defaults by customer
    customer_default = pd.crosstab(data['customerid'], data['no_default'])
    customer_default.columns = ['default_count', 'no_default_count']
    data = pd.merge(data, customer_default, how='left', on='customerid')

    # previous early and on time repayments
    data['early'] = np.where(data['term_delay'] < 0, 1, 0)
    data['on_time'] = np.where(data['term_delay'] == 0, 1, 0)

    # no. of early/on time payments by customer
    early_count = pd.crosstab(data['customerid'], data['early'])
    early_count.columns = ['0', 'early_count']
    data = pd.merge(data, early_count, how='left', on='customerid')

    on_time_count = pd.crosstab(data['customerid'], data['on_time'])
    on_time_count.columns = ['1', 'on_time_count']
    data = pd.merge(data, on_time_count, how='left', on='customerid')


    #previous number of loans taken
    prev_number = (data.groupby('customerid')['loannumber'].max()).reset_index(name='prev_loan_number')
    data = pd.merge(data, prev_number, how='left', on='customerid')

    # previous default rate
    data['default_rate'] = (data['default_count']/data['prev_loan_number']).round(2)

    # previous total amount of loans taken
    prev_total = (data.groupby('customerid')['loanamount'].sum()).reset_index(name='prev_total_loan')
    data = data = pd.merge(data, prev_total, how='left', on='customerid')

    # previous maximum amount of loans taken
    prev_amount = (data.groupby('customerid')['loanamount'].max()).reset_index(name='prev_max_loan_amount')
    data = pd.merge(data, prev_amount, how='left', on='customerid')

    return data

From the understanding of the previous loans dataframe, the following unique features were created for merging, analysis and possibly prediction:
- default_count - Number of times the customer has defaulted
- no_default_count - Number of times the customer has not defaulted
- early_count - Number of times the customer repaid early
- on_time_count - Number of times the customer repaid on time
- prev_loan_number - Total number of loans the customer has taken previously
- default_rate - The customer's previous default rate (default_count/prev_loan_number)
- prev_total_loan - Total amount of the loans the customer had previously taken
- prev_max_loan_amount - Maximum amount of a loan the customer had previously taken

In [ ]:
# create the dataframe to be used for merging
df_prevloans_merge = merge_prevloans(df_prevloans.copy())

In [ ]:
# drop unncessary columns
df_prevloans_merge = df_prevloans_merge.drop(columns=['systemloanid', 'loannumber', 'approveddate', 'creationdate', 'loanamount', 'totaldue', 'termdays',
                                                      'firstduedate', 'closeddate', 'firstrepaiddate', 'loan_duration', 'term_delay', 'no_default',
                                                      'early', 'on_time', '0', '1'])

In [ ]:
df_prevloans_merge = df_prevloans_merge.drop_duplicates(subset='customerid', ignore_index=True)

In [ ]:
df_prevloans_merge.shape

In [ ]:
df_prevloans_merge['customerid'].duplicated().sum()

The created dataframe df_prevloans_merge is now suitable for merging with the demographics and performance dataframes because all customer id's are unique.

#### For Analysis

In [ ]:
# features specific to previous loans
def features_prevloans(data):
    # apply customer default function
    data = loan_default(data)

    # merge demographics
    data = pd.merge(data, df_demographics, how='left', on='customerid')
    data = data.dropna()

    # repeat borrower
    data['repeat_borrower'] = np.where(data['loannumber'] > 1, 1, 0)

    return data

In [ ]:
# general features - that can also be applied to the merged data later on
def features_general(data):
    # date related
    # approval speed
    data['approval_speed_hours'] = (((data['approveddate'] - data['creationdate']).dt.seconds) / 3600).round()
    # approval hour
    data['approval_hour'] = data['approveddate'].dt.hour
    # approval day
    data['approval_day'] = data['approveddate'].dt.day_name()
    # loan year
    data['loan_year'] = data['creationdate'].dt.year
    # loan month
    data['loan_month'] = data['creationdate'].dt.month_name()

    # loan related
    # interest amount
    data['interest_amount'] = data['totaldue'] - data['loanamount']
    # interest rate
    data['interest_rate'] = (data['interest_amount'] / data['totaldue']).round(4)
    # loan amount category
    data['loan_amount_category'] = pd.cut(data['loanamount'], bins=[0, 10000.0, 30000.0, 60000.0], labels=['Small', 'Medium', 'Large'])
    # loan number category
    data['loan_number_category'] = pd.cut(data['loannumber'], bins=[0, 5, 15, 30], labels=['Low', 'Mid', 'High'])

    # client/demographics related
    # age
    data['age'] = data['creationdate'].dt.year - data['birthdate'].dt.year
    # age group
    data['age_group'] = pd.cut(data['age'], bins=[17, 25, 40, 55, 100], labels=['Young Adults', 'Adults', 'Middle Age', 'Seniors'])

    return data

From the understanding of the previous loans dataframe, the following features were created for analysis of past loans:
- no_default – Binary indicator showing whether the customer did not default on a loan: 1 = yes, 0 = no
- loan_year – The year when the loan was approved.
- loan_month – The month when the loan was approved.
- approval_hour – Hour of the day (0–23) when the loan was approved.
- approval_speed_hours – Time taken (in hours) between loan application creation and loan approval.
- approval_day – Day of the week when the loan was approved.
- interest_amount – Amount charged as interest for the loan.
- interest_rate – Percentage rate charged as interest, based on the loan amount.
- loan_amount_category – Categorical grouping of loan amounts (Small, Medium, Large).
- loan_number_category – Categorical grouping based on how many loans the customer has taken so far (Low, Mid, High)
- age – Customer’s age in years.
- age_group – Categorical grouping of customers based on age ranges (Young adults 18-25, Adults 26-40, Middle age 41-55, Seniors 56+).
- repeat_borrower – Binary indicator showing whether the customer has taken more than one loan: 1 = yes, 0 = no

In [ ]:
# apply
df_prevloans = features_prevloans(df_prevloans)
df_prevloans = features_general(df_prevloans)

# drop unnecessary columns
df_prevloans = df_prevloans.drop(columns=['systemloanid', 'approveddate', 'creationdate', 'closeddate', 'firstduedate', 'firstrepaiddate',
                                         'loan_duration', 'term_delay', 'birthdate'])

### Merging the Dataframes

In [ ]:
# merge df_perf and df_demographics
df_temp = pd.merge(df_perf, df_demographics, how='left', on='customerid')

In [ ]:
# merge df_temp and df_prevloans
df = pd.merge(df_temp, df_prevloans_merge, how='left', on='customerid')

### Feature Engineering - Main Dataframe

In [ ]:
# apply the general features function
df = features_general(df)

In [ ]:
# drop date columns
df = df.drop(columns=['approveddate', 'creationdate', 'birthdate'])

## Exploratory Data Analysis (EDA)
- Understand trends, patterns and  distributions from past loans
- Understand trends, patterns and distributions of the main dataframe

### EDA - Previous Loans

In [ ]:
#df_prevloans.to_csv('previous_loans.csv') - for power bi

In [ ]:
# check for missing values
df_prevloans.isna().sum().sum()

In [ ]:
# split into numerical, categorical and geographical columns
num_cols = ['loannumber', 'loanamount', 'totaldue', 'approval_speed_hours', 'approval_hour', 'interest_amount', 'interest_rate', 'age']

cat_cols = ['termdays', 'bank_account_type', 'bank_name_clients', 'employment_status_clients', 'loan_year', 'loan_month', 'approval_day',
            'loan_amount_category', 'loan_number_category', 'age_group', 'repeat_borrower']

geo_cols = ['longitude_gps', 'latitude_gps']

#### Numerical Columns

In [ ]:
# summary statistics
df_prevloans[num_cols].describe().round(2).T

##### **Insights:**
In the given dataset:
- The minimum number of loans a customer has taken is 1, the maximum is 26
- The minimum loan amount is 3000.0, the maximum is 60000.0
- The minimum as well as the most common/average approval is 1 hour, the maximum is 24 hours (1 day)
- The minimum approval hour is 00:00(12am), the maximum is 23:00(11pm).
- The minimum interest amount added to a loan is 225.0, the maximum is 9000.0
- The minimum interest rate is 0.02 percent, the maximum is 0.23 percent
- The minimum age is 21 years, the maximum is 56 years

In [ ]:
# correlation matrix
corr_matrix = df_prevloans[num_cols].corr()
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap='Blues', fmt='.3f')
plt.title('Correlation Matrix')
plt.show()

##### **Top 3 Strongest Positive Correlations (as one increases, the other increases):**
1. totaldue and loanamount = 0.995: Larger loan amounts result in higher total due
2. loanamount and loannumber = 0.795: Customers taking more loans also tend to borrow larger amounts.
3. totaldue and interest_amount = 0.788: As the total due increases, the interest amount also increases.

##### **Top 3 Strongest Negative Correlations (as one increases, the other decreases):**
1. loannumber and interest_rate = -0.446: The more loans a customer takes, the lower the interest rate tends to be, suggesting loyal or repeat customers receive better rates.
2. loanamount and interest_rate = -0.430: Higher loan amounts are mostly associated with lower interest rates
3. totaldue and interest_rate = -0.352: Customers who owe more overall (total due) generally face lower interest rates

The correlation matrix helps understand how the different numerical features interact with/affect each other

In [ ]:
# num_cols distribution and boxplot - loannumber, loanamount, age and approval speed
num_cols = ['loannumber', 'loanamount', 'approval_speed_hours', 'age']
for col in num_cols:
    plt.figure(figsize=(12, 4))
    # histogram
    plt.subplot(1, 2, 1)
    sns.histplot(df_prevloans[col], bins=30, kde=True)
    plt.title(f"{col.capitalize()} Distribution")
    plt.xlabel(col.capitalize())
    plt.ylabel('Count')

    #boxplot
    plt.subplot(1, 2, 2)
    sns.boxplot(x=df_prevloans[col], color='green')
    plt.title(f"{col.capitalize()} Boxplot")
    plt.xlabel(col.capitalize())
    plt.ylabel('Count')

    plt.tight_layout()
    plt.show()

##### **Insights:**
- The majority of customers took only one loan.
- A loan amount of 10000.0 was the most common among customers.
- On average, the banks approved loans within one hour of creation.
- Most customers were primarily in their 30s.

#### Categorical Columns

In [ ]:
# cat cols visualization
fig, axes = plt.subplots(6, 2, figsize=(12, 25))

for i, col in enumerate(cat_cols):
    ax = axes[i//2, i%2]
    sns.countplot(data=df_prevloans, x=col, ax=ax)
    ax.set_title(f"{col.capitalize()} Countplot")
    ax.set_xlabel(col.capitalize())
    ax.set_ylabel('Count')
    ax.tick_params(axis='x', rotation=90)
    axes[5,1].set_visible(False)

plt.tight_layout()
plt.show()

##### **Insights:**
- The most common term of a loan is 30 days. The least common term of a loan is 90  days
- The most common account type is a Savings account. The least common account type is a Current account
- GT Bank has the highest number of loan customers. Unity Bank has the lowest number of loan customers
- The employment status with the most customers is 'Permanent'. The employment status with the least customers is 'Contract'
- The month with the most loan activity is June. The month with the least loan activity is September
- The day with the most loan activity is Wednesday. The day with the least loan activity is Sunday
- Customers took more than one loan, with most having a small number of loans taken (0 - 5)
- Customers took smaller loan amounts (0 - 10000.0).
- Customers were mostly Adults (26 - 40 years)
- There are only 2 years in the dataset, 2016 and 2017. More loans were taken in 2017 than 2016


#### Interest Amount/Rate


In [ ]:
# check for differences in the interest amount
pd.set_option('display.max_rows', None)
df_prevloans.groupby(['loanamount', 'termdays', 'bank_name_clients'], observed=True )['interest_amount'].agg(['min', 'max'])

Different interest amounts were observed for the same amount + same term + same bank

In [ ]:
# check reason for different interest amounts/rates per bank

df_prevloans.groupby(['bank_name_clients', 'loanamount', 'termdays', 'age_group', 'employment_status_clients', 'bank_account_type', 'no_default',
                      'loannumber', 'loan_year', 'loan_month'], observed=True)['interest_amount'].agg(['min', 'max'])

In [ ]:
df_prevloans.loc[df_prevloans['loannumber'] == 1]

In [ ]:
df_prevloans.loc[df_prevloans['loannumber'] == 1, 'loanamount'].max()

In [ ]:
pd.reset_option('display.max_rows')

##### **Key Finding:**
The variation in interest rates among borrowers with the same loan amount and term can be explained by the interaction between loan number (loannumber) and repayment behavior (no_default)

It was observed that for:
1. Consistent Repayers (no_default = 1 or Low/0 default rate)
    - Interest rates decrease as loan number increases. This suggests banks use loan history as a trust and loyalty mechanism.
    - Borrowers who demonstrate reliability are rewarded with lower rates over time.
2. Defaulting Borrowers (no_default = 0 or High default rate)
    - Interest rates remain the same across loan numbers.
    - High number of past defaults prevent access to loyalty benefits.

Notably, the reduction in interest rate typically begins around the 4th loan, though this may vary depending on the customer’s repayment history (default rate).

This finding confirms the observed correlation between loannumber and interest_rate = -0.446

##### **Further Insights:**
- Interest rate increases as term days increases
- The higher the number of loans a customer has taken, the higher the loan amount the customer can borrow.
- The highest loan amount a customer can take without having a history with the bank is 10000.0

#### Rate of Defaults Based On Different Features

In [ ]:
# features to use for comparison
features = ['loannumber', 'age', 'age_group', 'termdays', 'loanamount', 'bank_account_type', 'bank_name_clients',
            'employment_status_clients', 'loan_month', 'loan_year', 'approval_day', 'repeat_borrower', 'approval_speed_hours', 'interest_amount',
            'loan_amount_category', 'loan_number_category']

In [ ]:
# visualise
fig, axes = plt.subplots(8, 2, figsize=(15, 28), constrained_layout=True)

for i, feat in enumerate(features):
    # calculate
    percent_target = df_prevloans.groupby([feat, 'no_default'], observed=True)['no_default'].count() / df_prevloans.groupby(feat, observed=True)['no_default'].count() * 100
    # change to a data frame
    percent_target = percent_target.reset_index(name='percentage')
    # get dataframe of bad flags only
    default_percent = percent_target[percent_target['no_default'] == 0]

    #plot
    ax = axes[i//2, i%2]
    sns.barplot(data=default_percent, x=feat, y='percentage', ax=ax)
    ax.set_title(f"Default Rate per {feat.capitalize()}")
    ax.set_xlabel(feat.capitalize())
    ax.set_ylabel('Default Rate')
    ax.tick_params(axis='x', rotation=90)



plt.show()

##### **Insights:**
- People defaulted more on their 2nd loans, then their 11th. Lowest: 20 - 26 with 0%. Having a high loan number (20 - 26) is only possible with a good repayment history.
- Clients aged 22 years had the highest default rate, followed by 43 years. Lowest: 56 years with 0% (one person). Next Lowest:55 years
- Clients in the Young Adults age group (18 - 25 years) had the highest default rate. Lowest: Seniors (56 years and above). Next Lowest: Adults (26 - 40 years)
- Loans with a 30 day term had the highest default rates, followed by 15 days. Lowest: 90 days with 0%. A longer term alludes to a higher loan amount, higher loan number and a good repayment history.
- Loan amounts of 9000 had the highest default rate, then 4000. Lowest: 60,000. A customer can only take a higher loans if they have a good repayment history of the smaller loans
- Clients with a Savings account had the highest default rate, followed by Other. Lowest: Current
- Sterling Bank had the highest default rate, then Eco Bank. Lowest: Unity Bank
- Unemployed clients had the highest default rate, then Self-employed. Lowest: Contract workers (one person), Next Lowest: Retired workers.
- The month of September had the highest default rate, then May. Lowest: July. September is a known month for school resumption, this could have affected some customers ability to pay back.
- The year 2016 had a higher default rate. Upon researching, I found out that the year 2016 had significant financial problems, a major one being a global economic slowdown, with world GDP growth dropping to 2.3%, down from 2.7% the previous year. This could have affected customers abilities to pay back
- Approval speeds of 20 hours and 24 hours had the highest default rate. This suggests that higher approval speeds give a higher chance of default.
- Smaller loan amounts had the highest default rates. Considering banks only allow higher loans for repeat customers with good repayment history, this is to be expected. Smaller loan amounts are like the 'starting points' for loans.
- Customers with a low loan number (0-5) had the highest default rates.

#### Geographical Columns

In [ ]:
# get data from prevloans to use for geographic analysis
df_geo = df_prevloans.loc[:, geo_cols + ['no_default']]

In [ ]:
# create a geo dataframe
df_geo = gpd.GeoDataFrame(df_geo, geometry=gpd.points_from_xy(df_geo.longitude_gps, df_geo.latitude_gps))

In [ ]:
df_geo.head()

In [ ]:
df_geo.shape

In [ ]:
# get world country data geojson
url = 'https://raw.githubusercontent.com/martynafford/natural-earth-geojson/refs/heads/master/110m/cultural/ne_110m_admin_0_countries.json'
world = gpd.read_file(url)

In [ ]:
# rename columns to match 'naturalearth_lowres'
rename_mapper = {
    'POP_EST': 'pop_est',
    'CONTINENT': 'continent',
    'NAME': 'name',
    'ADM0_A3': 'iso_a3',
    'GDP_MD_EST': 'gdp_md_est'
}
world_data = world.rename(columns=rename_mapper)

# keep only the columns that naturalearth_lowres had
world_data = world_data.loc[:, ['pop_est', 'continent', 'name', 'iso_a3', 'gdp_md_est', 'geometry']]

world_data.head()

In [ ]:
# set crs of the dataframe to match world_data
df_geo = df_geo.set_crs(epsg=4326)

In [ ]:
# check continents contained in the loan data
df_geo_continents = gpd.sjoin(df_geo, world_data[['continent', 'geometry', 'name', 'iso_a3']], how='inner')
df_geo_continents['continent'].value_counts()

In [ ]:
# check countries contained in the loan data and their default counts
geo_countries = pd.crosstab([df_geo_continents['continent'], df_geo_continents['name']], df_geo_continents['no_default'])
geo_countries

Considering there are more good flags than bad flags (no default vs default) in the data as a whole, i'll be focusing on the defaults alone

In [ ]:
# create a function to calculate default rates on a crosstab df
def crosstab_data(df_crosstab):
    # reset index
    df = df_crosstab.reset_index()

    # calculate total loan count and default rate
    df['total_count'] = df[0] + df[1]  # 1 = no default, 0 = default
    df['default_count'] = df[0]
    df['no_default_count'] = df[1]
    df['default_rate'] = df[0] / df['total_count']
    df['default_rate_pct'] = df['default_rate'] * 100

    # clean column names
    df = df.drop(columns=[0, 1])

    return df

In [ ]:
geo_countries = crosstab_data(geo_countries)
geo_countries

In [ ]:
# get continent data to plot choropleth map on default rates (percentage)
continent_choropleth = world_data.merge(geo_countries, on = ['continent', 'name'], how='left')

In [ ]:
# change to geojson
continents_geojson = continent_choropleth.__geo_interface__

In [ ]:
# plot total loans taken per country
fig = px.choropleth(
    continent_choropleth,
    geojson=continents_geojson,
    locations=continent_choropleth.index,
    color= 'total_count',
    hover_name= 'name',
    hover_data={
        'continent': True,
        'total_count': True,
    },
    color_continuous_scale='Teal',
    title='Geographical Distribution of Loans Taken'
)
fig.show()

Hover over the map to view country name, continent and loan count

In [ ]:
# plot default rates (percentage) per country
fig = px.choropleth(
    continent_choropleth,
    geojson=continents_geojson,
    locations=continent_choropleth.index,
    color= 'default_rate_pct',
    hover_name= 'name',
    hover_data={
        'continent': True,
        'total_count': True,
        'default_count': True,
        'default_rate_pct': ':.2f'
    },
    color_continuous_scale='Teal',
    title='Geographical Distribution of Default Rate'
)
fig.show()

Hover over the map to view country name, continent, loan count, default count and default rate (percentage)

##### **Statistics/Insights:**
There are 6 coordinates and 37 loans without a continent/country match in the world data.

From the 12,273 loans that do have a match, I have inferred that customers are located in 5 continents at different concentrations in different countries with the most number of customers being from Nigeria, Africa.

The continents and the total number of loans taken and the breakdown per country are as follows: (all default rates are rounded to the nearest whole number excluding 0 defaults, 100% defaults or unless stated otherwise)

Africa with 12,176 total loans divided into:
  - Nigeria - 12,137 total loans, 2506 defaults, 21% default rate
  - Ghana - 18 total loans, 4 defaults, 22% default rate
  - Benin - 10 total loans, 1 default, 10% default rate (exact)
  - Côte d'Ivoire - 5 total loans, 1 default, 20% default rate (exact)
  - Niger - 4 total loans, 3 defaults, 75% default rate (exact)
  - Cameroon - 2 total loans, 0 defaults, 0% default rate
    
North America with 55 loans divided into:
  - United States of America - 55 loans, 13 defaults, 24% default rate
    
Europe with 25 loans divided into:
  - United Kingdom - 15 loans, 8 defaults, 53% default rate
  - Russia - 10 loans, 0 defaults, 0% default rate
    
Asia with 10 loans divided into:
  - United Arab Emirates - 5 loans, 2 defaults, 40% default rate (exact)
  - Thailand - 4 loans, 0 defaults, 0% default rate
  - China - 1 loan, 1 default, 100% default rate
    
Oceania with 7 loans divided into:
  - Australia - 7 loans, 1 default, 14% default rate

Apart from Nigeria, the number of loans taken per country is too low for a reliable inference on default rates to be gotten or to get deeper insights like default by region or default by state. However the above information can be used to make decisions such as no longer accepting loans from Niger based on the business goals or preference.

I will now provide further insights on default rates in Nigeria.

In [ ]:
# get world regional data - admin_1
url2 = 'https://raw.githubusercontent.com/martynafford/natural-earth-geojson/refs/heads/master/10m/cultural/ne_10m_admin_1_states_provinces.json'
admin_1 = gpd.read_file(url2)

In [ ]:
# filter for Nigeria only
nigeria_admin1 = admin_1[admin_1['admin'] == 'Nigeria']

In [ ]:
# create geodataframe
nigeria_gdf = (gpd.sjoin(df_geo, nigeria_admin1[['name', 'admin', 'adm1_code', 'postal', 'geometry']], how='left'))
nigeria_gdf = nigeria_gdf.set_geometry(df_geo.geometry)

In [ ]:
# rows that didn’t match any state in the data
missing = nigeria_gdf[nigeria_gdf['name'].isna()].drop_duplicates()

print(f"There are {len(missing)} locations without a state match, including the initial 37")
print(missing[['longitude_gps','latitude_gps']].head(20))

In [ ]:
# add region column

# create a dictionary with all regions
region_map = {
    "North West": ["Kano", "Kaduna", "Katsina", "Kebbi", "Jigawa", "Sokoto", "Zamfara"],
    "North East": ["Borno", "Adamawa", "Yobe", "Taraba", "Bauchi", "Gombe"],
    "North Central": ["Kogi", "Nasarawa", "Benue", "Niger", "Kwara", "Plateau", "FCT Abuja"],
    "South West": ["Lagos", "Oyo", "Ogun", "Ondo", "Osun", "Ekiti"],
    "South East": ["Anambra", "Imo", "Abia", "Ebonyi", "Enugu"],
    "South South": ["Edo", "Delta", "Bayelsa", "Rivers", "Akwa Ibom", "Cross River"],
}

# create a function that maps the region to a state
def get_region(state):
    for region, states in region_map.items():
        if state in states:
            return region
    return None

In [ ]:
# add region to nigeria gdf
nigeria_gdf['region'] = nigeria_gdf['name'].apply(get_region)

In [ ]:
# set crs
nigeria_gdf = nigeria_gdf.set_crs(epsg=4326)

In [ ]:
nigeria_gdf.head()

In [ ]:
# default rate per state
nigeria_states = pd.crosstab([nigeria_gdf['name'], nigeria_gdf['region']], nigeria_gdf['no_default'])
nigeria_states = crosstab_data(nigeria_states)
nigeria_states.columns = ['name', 'region', 'total_count', 'default_count', 'no_default_count', 'default_rate', 'default_rate_pct']
nigeria_states

In [ ]:
# default rate per region
nigeria_regions = pd.crosstab(nigeria_gdf['region'], nigeria_gdf['no_default'])
nigeria_regions = crosstab_data(nigeria_regions)
nigeria_regions.columns = ['region', 'total_count', 'default_count', 'no_default_count', 'default_rate', 'default_rate_pct']
nigeria_regions

In [ ]:
# get state data to plot choropleth map on default rates (percentage)
states_choropleth = nigeria_admin1.merge(nigeria_states, on = 'name', how='left')

In [ ]:
# change to geojson
nigeria_states_geojson = states_choropleth.__geo_interface__

In [ ]:
# plot total loans per state in Nigeria
fig = px.choropleth(
    states_choropleth,
    geojson=nigeria_states_geojson,
    locations=states_choropleth.index,
    color= 'total_count',
    hover_name= 'name',
    hover_data={
        'total_count': True,
    },
    title='Loan Distribution by States in Nigeria',
    color_continuous_scale='Greens',
)
fig.update_geos(fitbounds='locations', visible=False)
fig.show()

Hover over the map to view state name and loan count

In [ ]:
# plot default rates (percentage) per state in Nigeria
fig = px.choropleth(
    states_choropleth,
    geojson=nigeria_states_geojson,
    locations=states_choropleth.index,
    color= 'default_rate_pct',
    hover_name= 'name',
    hover_data={
        'total_count': True,
        'default_count': True,
        'default_rate_pct': ':.2f'
    },
    title='Loan Default Rate by States in Nigeria',
    color_continuous_scale='Greens',
)
fig.update_geos(fitbounds='locations', visible=False)
fig.show()

Hover over the map to view state name, loan count, default count and default rate (percentage)

In [ ]:
# get regional data to plot choropleth map on default rates (percentage)
nigeria_regions_gdf = nigeria_gdf.dissolve(by='region', as_index=False)

In [ ]:
# map region to state
nigeria_admin1.loc[:, 'region'] = nigeria_admin1['name'].apply(get_region)

In [ ]:
# aggregate shapes
regions_admin1 = nigeria_admin1.dissolve(by='region', aggfunc='sum')

In [ ]:
# merge statistical data such as total_count
regions_choropleth = regions_admin1.merge(nigeria_regions, how='left', on = 'region')

In [ ]:
# convert to geojson
nigeria_regions_geojson = regions_choropleth.__geo_interface__

In [ ]:
# plot total loans per region in Nigeria
fig = px.choropleth(
    regions_choropleth,
    geojson=nigeria_regions_geojson,
    locations=regions_choropleth.index,
    color= 'total_count',
    hover_name= 'region',
    hover_data={
        'total_count': True,
    },
    title='Loan Distribution by Regions in Nigeria',
    color_continuous_scale='Greens',
)
fig.update_geos(fitbounds='locations', visible=False)
fig.show()

Hover over the map to view region name and loan count

In [ ]:
# plot default rates (percentage) per region in Nigeria
fig = px.choropleth(
    regions_choropleth,
    geojson=nigeria_regions_geojson,
    locations=regions_choropleth.index,
    color= 'default_rate_pct',
    hover_name= 'region',
    hover_data={
        'total_count': True,
        'default_count': True,
        'default_rate_pct': ':.2f'
    },
    title='Loan Default Rate by Regions in Nigeria',
    color_continuous_scale='Greens',
)
fig.update_geos(fitbounds='locations', visible=False)
fig.show()

Hover over the map to view region name, loan count, default count and default rate (percentage)

##### **Statistics:**
Nigeria has the highest number of customers taking loans in the dataset. There are 145 locations without a state match, including the initial 37. From the matched coordinates in the data:
- The region with the most customers is the South West Region - 7747 loans, the lowest being North East - 95 loans.
- The state with the most customers is Lagos State - 5209 loans, the lowest being Zamfara and Jigawa - 3 loans.

The regions, total number of loans taken and the breakdown per state is as follows: (all default rates are rounded to the nearest whole number excluding 0 defaults, 100% defaults or unless stated otherwise)

**South West - 7747 loans, 1610 defaults, 21% default rate**
- Lagos - 5209 loans, 1050 defaults, 20% default rate
- Oyo - 1039 loans, 246 defaults, 24% default rate
- Osun - 226 loans, 54 defaults, 24% default rate
- Ondo - 279 loans, 41 defaults, 15% default rate
- Ogun - 898 loans, 192 defaults, 21% default rate
- Ekiti - 96 loans, 27 defaults, 28% default rate

**South South - 1330 loans, 308 defaults, 23% default rate**
- Rivers - 477 loans, 109 defaults, 23% default rate
- Edo - 155 loans, 42 defaults, 27% default rate
- Delta - 309 loans, 54 defaults, 17% default rate
- Cross River - 108 loans, 33 defaults, 31% default rate
- Bayelsa - 190 loans, 50 defaults, 26% default rate
- Akwa Ibom - 91 loans, 20 defaults, 22% default rate

**North Central - 684 loans, 123 defaults, 18% default rate**
- Plateau - 109 loans, 17 defaults, 16% default rate
- Niger - 71 loans, 8 defaults, 11% default rate
- Kwara - 369 loans, 70 defaults, 19% default rate
- Kogi - 47 loans, 11 defaults, 23% default rate
- Benue - 88 loans, 17 defaults, 19% default rate

**North West - 431 loans, 71 defaults, 16% default rate**
- Zamfara - 3 loans, 0 defaults, 0% default rate
- Sokoto - 19 loans, 2 defaults, 11% default rate
- Jigawa - 3 loans, 0 defaults, 0% default rate
- Kebbi - 4 loans, 0 defaults, 0% default rate
- Katsina - 17 loans, 5 defaults, 29% default rate
- Kano - 103 loans, 14 defaults, 14% default rate
- Kaduna - 282 loans, 50 defaults, 18% default rate

**South East - 382 loans, 83 defaults, 22% default rate**
- Imo - 82 loans, 15 defaults, 18% default rate
- Enugu - 124 loans, 20 defaults, 16% default rate
- Ebonyi - 19 loans, 6 defaults, 32% default rate
- Anambra - 101 loans, 26 defaults, 26% default rate
- Abia - 56 loans, 16 defaults, 29% default rate

**North East - 95 loans, 17 defaults, 18% default rate**
- Gombe - 15 loans, 3 defaults, 20% default rate (exact)
- Borno - 5 loans, 1 default, 20% default rate (exact)
- Bauchi - 23 loans, 3 defaults, 13% default rate
- Adamawa - 52 loans, 10 defaults, 19% default rate

**Higest/Lowest:**
- The region with the highest default rate is South South - 1330 loans, 308 defaults, 23% default rate.
- The region with the lowest default rate is North West - 431 loans, 71 defaults, 16% default rate

- The state with the highest default rate is Ebonyi - 19 loans, 6 defaults, 32% default rate.
- The state with the lowest default rate excluding those at 0% due to the low number of loans is Niger - 71 loans, 8 defaults, 11% default rate and Sokoto - 19 loans, 2 defaults, 11% default rate

##### **Insights:**

**Why Lagos/South West leads:**
- Lagos is Nigeria's commercial capital and largest city (~15M people)
- Most banks, fintechs, and financial services are headquartered there
- Better internet, mobile penetration for digital lending
- Higher population density = more potential customers
- Generally higher disposable income supports loan demand

**Regional Default Rate Patterns**

**South South (23% default rate) - Highest Risk**
- Economy heavily reliant on oil sector volatility
- Oil spills, pollution affecting agriculture/fishing
- Despite oil wealth, limited economic diversification
- A state like Cross River (31%) is a border state with limited industrial base

**North West (16% default rate) - Lowest Risk**
- More stable, seasonal income from farming
- Islamic banking principles, cautious borrowing
- Less exposure to volatile urban employment
- Social capital for loan repayment support

 **State-Level Insights**

**High-Risk States:**
- Ebonyi (32%): Predominantly rural, limited industry, relies on agriculture
- Cross River (31%): Border state, tourism-dependent, limited diversification  
- Katsina (29%): Despite being northern, faces security challenges affecting economy
- Abia (29%): Industrial decline, unemployment in manufacturing sectors

**Low-Risk States:**
- Niger (11%): Federal Capital Territory proximity, government employment stability
- Sokoto (11%): Strong Islamic financial culture, agricultural stability
- Ondo (15%): Cocoa production, more diversified agriculture

**Business Implications**

**Volume vs Risk Trade-offs:**
- Lagos: Massive volume (43% of loans) with moderate risk (20%) = profit center
- South East: Higher risk but smaller volume = manageable with targeted strategies  
- North: Lower risk but limited volume = expansion opportunity

**Regional Strategies:**
- South West: Focus on volume growth with current risk levels
- South South: Implement stricter underwriting given oil volatility
- North: Develop Sharia-compliant products, expand rural access
- South East: Partner with local institutions for better risk assessment

**Cultural Considerations:**
- Northern states: Islamic finance principles, community-based lending
- Southern states: More Western financial practices, individual credit focus
- Oil states: Income volatility requires flexible repayment terms


#### **Final Conclusion on Previous Loans EDA:**
The data shows that defaults aren’t random; they revolve around new borrowers, smaller loan amounts, shorter terms, smaller loan numbers and repayment history (for repeat customers). In contrast, stability, loyalty, and stronger financial profiles lower the likelihood of default. Location also has a role to play as well due to regional economic conditions, infrastructure development, and cultural financial practices.

Borrower demographics and repayment behavior act as the most direct predictors, while bank policies shape these outcomes indirectly through mechanisms like risk-based interest rates and loyalty incentives. Together, these interactions create patterns that a default prediction model can capture, allowing banks not only to forecast risk with greater accuracy but also to refine their lending strategies in ways that support both growth and repayment stability.

### EDA - Main Dataframe

In [ ]:
# check for missing values
df.isna().sum().sum()

In [ ]:
# percentage of missing values
df.isna().sum().sort_values(ascending=False)/len(df) * 100

In [ ]:
# loan months
df.loan_month.unique()

In [ ]:
# loan year
df.loan_year.unique()

In [ ]:
df.dtypes

#### **Observations:**
- There are approximately 25% of customers missing all their demographic information due to there not being a match for their customerid in the demographics dataframe. The 36% missing in the employment_status_clients column includes those without a match as well as those that were missing in the original demographics dataframe
- There are approximately 0.2% of customers missing previous loans information
- The counts and age are of the wrong data type - float, it should be integer
- The termdays should be categorical for better interpretability
- There is no distinct difference between customers with missing demographic information and those without missing demographic information
- All loans are from the year 2017
- All loans are from the month of July

#### **Insights:**
- 1095 customers (approximately 25%) are missing all demographic data
- Considering that:
  - all loans are from July, 2017,
  - there are other customers with their full demographic information,
  - all the customers have taken a loan before,
  - and a loan can't be taken without submitting your birth date. It can be concluded that the missingness is most likely due to bank issues e.g data transfer/storage. This can be best described as MAR (Missing At Random) since the missing value can be tied to an observed variable.
-  The rows of the customers missing previous loans information can be dropped

#### **To Do:**
- Create a copy of the main dataframe with missing values as imputing values will create a bias and disrupt the integrity of the analysis
- Change incorrect data types
- Perform EDA on clean data

In [ ]:
#eda was also done on the full data to ensure correct values for categorical data, summary statistics etc.

In [ ]:
# analysis dataframe
df1 = df.copy()
df1 = df1.dropna()

In [ ]:
df1.isna().sum().sum()

In [ ]:
# change data types
dtype_cols = ['default_count', 'no_default_count', 'early_count', 'on_time_count', 'prev_loan_number', 'approval_speed_hours', 'age']
for col in dtype_cols:
    df1[col] = df1[col].astype(int)

df1['termdays'] = df1['termdays'].astype('category')

In [ ]:
# split into numerical, categorical and geographical columns
num_cols = ['loannumber', 'loanamount', 'totaldue', 'default_count', 'no_default_count', 'early_count', 'on_time_count', 'prev_loan_number',
            'default_rate', 'prev_total_loan', 'prev_max_loan_amount', 'approval_speed_hours', 'approval_hour', 'interest_amount', 'interest_rate', 'age']

cat_cols = ['termdays', 'bank_account_type', 'bank_name_clients', 'employment_status_clients', 'approval_day',
            'loan_amount_category', 'loan_number_category', 'age_group']

#### Numerical Columns

In [ ]:
# summary statistics
df1[num_cols].describe().round(2).T

##### **Insights**:
In the given dataset:
- The minimum number of loans a customer has taken is 2, the maximum is 27. This suggests that all the customers have taken a loan before
- The minimum loan amount is 10000.0, the maximum is 60000.0
- The minimum default count is 0, the maximum is 11 (full data).
- The minimum no default count is 0, the maximum is 26
- The minimum number of loans previously taken by a customer is 1, the maximum is 26
- The minimum default rate is 0.00%, the maximum is 1.00%
- The minimum as well as the most common/average approval speed is 1 hour, the maximum is 19 hours
- The minimum approval hour is 00:00(12am), the maximum is 23:00(11pm).
- The minimum interest amount added to a loan is 0.0, the maximum is 9000.0
- The minimum interest rate is 0.0 percent, the maximum is 0.23 percent
- The minimum age is 21 years, the maximum is 56 years

This was also done on the full data to get the actual values, 'full data' was used to show that though the max on the summary statistics of default count is 9 on the clean data, the actual max is 11.

In [ ]:
pd.set_option('display.max_columns', None)
df1[df1['interest_amount'] == 0.0]

This is the customer's 12th loan and the waive in interest can be attributed to the customer defaulting on only 1 out of 11 past loans. 7 payments were early and 3 were on time. This correlates with the findings in the previous loans EDA. The customer has a permanent employment status which had the 4th lowest default rate of all 6 in the category and is in the adult age group which had the 3rd lowest default of all 4 in the category

In [ ]:
# correlation matrix
corr_matrix = df1[num_cols].corr()
plt.figure(figsize=(15, 10))
sns.heatmap(corr_matrix, annot=True, cmap='Blues', fmt='.3f')
plt.title('Correlation Matrix')
plt.show()

##### **Top 3 Strongest Positive Correlations (as one increases, the other increases):**
1. totaldue and loanamount = 0.994: Larger loan amounts result in higher total due
2. no_default_count and early_count = 0.977: Borrowers who never or hardly default often repay early, showing strong reliability.
3. loannumber/prev_loan_number and prev_total_loan = 0.946: Customers with more loan cycles typically accumulate higher past loan amounts, reflecting loyalty and borrowing capacity.

##### **Top 3 Strongest Negative Correlations (as one increases, the other decreases):**
1. no_default_count and interest_rate = -0.583: The more defaults avoided, the lower the interest rate offered.
2. loannumber/prev_loan_number and interest_rate = -0.571: Repeat borrowers tend to secure lower rates.
3. prev_total_loan and interest_rate = -0.569: Borrowers with high loan numbers tend to have reduced interest rates.

In [ ]:
# num_cols distribution and boxplot - - loannumber, loanamount, age and approval speed
num_cols = ['loannumber', 'loanamount', 'approval_speed_hours', 'age']
for col in num_cols:
    plt.figure(figsize=(12, 4))
    # histogram
    plt.subplot(1, 2, 1)
    sns.histplot(df1[col], bins=30, kde=True)
    plt.title(f"{col.capitalize()} Distribution")
    plt.xlabel(col.capitalize())
    plt.ylabel('Count')

    #boxplot
    plt.subplot(1, 2, 2)
    sns.boxplot(x=df1[col], color='green')
    plt.title(f"{col.capitalize()} Boxplot")
    plt.xlabel(col.capitalize())
    plt.ylabel('Count')

    plt.tight_layout()
    plt.show()

##### **Insights:**
- Most customers are on their 2nd loan
- Most customers took a loan amount of 10000
- Bank usually take 1 hour to approve loans after creation
- The customers are majorly in their 30's

#### Categorical Columns

In [ ]:
# cat cols visualization
fig, axes = plt.subplots(4, 2, figsize=(12, 15))

for i, col in enumerate(cat_cols):
    ax = axes[i//2, i%2]
    sns.countplot(data=df1, x=col, ax=ax)
    ax.set_title(f"{col.capitalize()} Countplot")
    ax.set_xlabel(col.capitalize())
    ax.set_ylabel('Count')
    ax.tick_params(axis='x', rotation=90)

plt.tight_layout()
plt.show()

##### **Insights:**
- The most common term of a loan is 30 days. The least common term of a loan is 90 days (higher term = higher loan amount = higher loan number)
- The most common account type is a Savings account. The least common account type is a Current account
- GT Bank has the highest number of loan clients. Unity Bank has the lowest number of loan clients
- The employment status with the most clients is 'Permanent'. The employment status with the least clients is 'Contract'
- The day with the most loan activity is Wednesday (full data). The day with the least loan activity is Sunday
- Customers tend to take more than one loan, with most having a small number of loans taken (0 - 5)
- Customers take smaller loan amounts (0 - 10000.0).
- Customers were mostly Adults (26 - 40 years)

This was also done on the full data to get the actual values, 'full data' is used to show that though the day with the most activity shows Monday, it is actually Wednesday

In [ ]:
# target column imbalance
sns.countplot(data=df, x='good_bad_flag')
plt.title(f"Target Countplot")
plt.xlabel('Flag')
plt.ylabel('Count')
plt.show()

In [ ]:
df.good_bad_flag.value_counts()

In [ ]:
df.good_bad_flag.value_counts(normalize=True) * 100

The dataset shows a significant class imbalance with 78.2% (3416) good flags versus 21.8% (952) bad flags. This imbalance would require careful handling in predictive modeling efforts through techniques like SMOTE, class weighting, or stratified sampling as it will cause a bias towards the majority class (in this case, good flags).

#### **Final Conclusion on Main Dataframe EDA:**
1. **Key Findings and Validation:**

With only 4,368 rows in this dataset, the size is too small for a detailed analysis such as default rate analysis across multiple features or geographical analysis as we did in previous loans. Despite these limitations, I was able to extract meaningful insights by leveraging knowledge from previous loan analysis and conducting comprehensive correlation and distribution analysis. The current analysis confirms and builds upon the following critical insights from previous loan data:
- Loyalty Based Pricing Mechanism: The dataset validates behavioral incentives, showing that interest rates decrease for consistent repayers (correlation = -0.583 between no_default_count and interest_rate)
- Loan Cycle Benefits: Repeat borrowers (higher loan numbers) consistently secure lower interest rates (correlation = -0.571), confirming the reward system for customer loyalty
- Risk Segmentation: Previous findings about high-risk demographics (young adults, unemployed individuals) and seasonal patterns (September defaults) are reinforced

2. **Operational Insights:**
- Loan Processing Efficiency: Most loans are approved within 1 hour, indicating streamlined operations
- Customer Demographics: Primarily adults aged 26-40 with permanent employment and savings accounts through GT Bank

#### **Recommendations**
Given the dataset limitations, recommendations focus on operational improvements leveraging proven loyalty and behavioral patterns:

1. **Data-Driven Risk Management:**
   - Loyalty Based Pricing: Formalize the observed pattern by reducing interest rates after successful loan milestones (4th, 5th loans) for consistent repayers while maintaining flat rates for customers with default history
   - Multi-factor Assessment: Combine loan history + repayment behavior + geographic risk + demographic factors (age, employment, account type) for comprehensive risk evaluation
   - Data Collection Enhancement: Expand data collection efforts to build a more robust dataset (target 10,000+ entries) for reliable statistical analysis and predictive modeling

2. **Operational Implementation:**
   - Automated Systems: Build loyalty tracking systems to automatically adjust rates and limits based on successful loan completion milestones
   - Real-time Monitoring: Implement tracking of validated risk factors (age groups, employment status, seasonal patterns) identified in previous analysis
   - Seasonal Adjustments: Prepare for September default spikes by implementing additional screening during August loan originations

3. **Strategic Portfolio Management:**
   - Predictive Scoring: Use identified patterns (new borrowers + small amounts + short terms + volatile regions) for proactive default prevention
   - Segment Focus: Leverage confirmed low-risk segments (seniors, contract workers, established customers) while carefully managing exposure to high-risk categories
   - Future Model Development: Once larger datasets are available, develop integrated predictive models incorporating all behavioral and demographic insights

While the current dataset of 4,368 rows provides valuable directional insights and confirms previously identified patterns, it falls short of the statistical power needed for robust predictive modeling or categorical analysis.



#### **Predictive Modeling Approach**

Despite the dataset size limitations, I will proceed with building a predictive model as part of this project. The model development will focus on:
-  Class Imbalance Handling: Address the 78.2% vs 21.8% target distribution using techniques like SMOTE, class weighting, or stratified sampling
- Model Tuning: Implement strategies to improve model performance

Expected Outcomes:
- The model will help predict loan defaults
- Performance metrics will be interpreted cautiously due to dataset size constraints
- Results will serve as a foundation for future model enhancement with larger datasets

The current analysis provides a solid foundation for model features, and while prediction accuracy may be limited by sample size, the model will demonstrate the methodology and framework for larger scale implementation.

## Data Preparation

In [ ]:
df.columns

In [ ]:
# drop columns that won't be useful for prediction
df = df.drop(['customerid', 'systemloanid', 'totaldue', 'longitude_gps', 'latitude_gps', 'loan_year', 'loan_month'], axis=1)

In [ ]:
# missing values
(df.isna().sum().sort_values(ascending=False)/len(df))*100

In [ ]:
# drop rows with less than 5% missing values
df = df.dropna(subset=['default_count', 'no_default_count', 'early_count',
                       'on_time_count', 'prev_loan_number', 'default_rate', 'prev_total_loan',
                       'prev_max_loan_amount'], ignore_index=True)

In [ ]:
# data types
df.dtypes

In [ ]:
# change data types
dtype_cols = ['default_count', 'no_default_count', 'early_count', 'on_time_count', 'prev_loan_number', 'approval_speed_hours']
for col in dtype_cols:
    df[col] = df[col].astype(int)

df['termdays'] = df['termdays'].astype('category')

In [ ]:
# encode target
df['good_bad_flag'] = np.where(df['good_bad_flag'] == 'Good', 1, 0) # Good = 1, Bad = 0

In [ ]:
# add missingness indicator - could be a useful feature
null_cols = [col for col in df if df[col].isna().sum() > 0]
null_cols
for col in null_cols:
    df[f"{col}_is_missing"] = np.where(df[col].isna(), 1, 0)

In [ ]:
# split data into features (X) and target(y)
X = df.drop('good_bad_flag', axis=1)
y = df['good_bad_flag']

In [ ]:
# split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.08, stratify=y, random_state=20)

The test size was reduced to improve model performance

## Data Preprocessing

- Handle missing values
- Feature selection
- Scaling and encoding

### Handling Missing Values
Creating imputers for missing values using model prediction

In [ ]:
# create functions for preprocessing and prediction

# preprocess data
def preprocess(data):
    #scale
    for num_col in data.select_dtypes(include=np.number):
        if num_col == 'is_na':
            pass
        else:
            scaler = StandardScaler()
            data[num_col] = scaler.fit_transform(data[[num_col]])
    #encode
    for cat_col in data.select_dtypes(exclude=np.number):
        encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        encoded = encoder.fit_transform(data[[cat_col]])
        encoded = pd.DataFrame(encoded, index=data.index, columns=encoder.get_feature_names_out([cat_col]))
        data = data.drop(cat_col, axis=1).join(encoded)
    return data

# split data
def split(data, col):
    # split into train and test
    train = data[data['is_na'] == 0]
    test = data[data['is_na'] == 1]
    # X_train, y_train, X_test
    X_train = train.drop([col, 'is_na'], axis=1)
    y_train = train[col]
    X_test = test.drop([col, 'is_na'], axis=1)
    return X_train, y_train, X_test

# predict missing values
def train_predict_cat(X_train, y_train, X_test):
    model = RandomForestClassifier(random_state=20)
    # fit and predict
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return y_pred

def train_predict_num(X_train, y_train, X_test):
    model = RandomForestRegressor(random_state=20)
    # fit and predict
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return y_pred

In [ ]:
# create categorical imputer
class CatImputer(TransformerMixin, BaseEstimator):
    #fit
    def fit(self, X, y=None):
        self.feature_names_in_ = X.columns.tolist()
        return self
    #transform - impute
    def transform(self, X):
        df = X.copy().reset_index(drop=True)

        # get categorical columns
        cat_cols = df.select_dtypes(exclude=np.number).columns.tolist()

        for col in cat_cols:
            # get index of missing values
            nan_mask = df[col].isna()
            if not nan_mask.any():
                    continue

            # add column to indicate missing
            df['is_na'] = nan_mask.astype(int)

            # preprocess
            X_temp = df.drop([col], axis=1)
            y_temp = df[col]
            X_temp = preprocess(X_temp)
            data = X_temp.join(y_temp)

            # X_train, y_train, X_test
            X_train, y_train, X_test = split(data, col)

            if len(X_train) > 0 and len(X_test) > 0:
                # train model and predict
                y_pred = train_predict_cat(X_train, y_train, X_test)

                # impute
                df.loc[nan_mask, col] = y_pred

            df.drop(columns='is_na', inplace=True)
        return df

        def get_feature_names_out(self, input_feat = None):
            if input_feat is None:
                return np.array(self.feature_names_in_)
            else:
                return np.array(input_feat)

In [ ]:
# create numerical imputer
class NumImputer(TransformerMixin, BaseEstimator):
    #fit
    def fit(self, X, y=None):
        self.feature_names_in_ = X.columns.tolist()
        return self

    #transform - impute
    def transform(self, X):
        df = X.copy().reset_index(drop=True)

        num_cols = df.select_dtypes(include=np.number).columns.tolist()
        for col in num_cols:
            # get index of missing values
            nan_mask = df[col].isna()
            if not nan_mask.any():
                    continue

            # add column to indicate missing
            df['is_na'] = nan_mask.astype(int)

            # preprocess
            X_temp = df.drop([col], axis=1)
            y_temp = df[col]
            X_temp = preprocess(X_temp)
            data = X_temp.join(y_temp)

            # X_train, y_train, X_test
            X_train, y_train, X_test = split(data, col)

            if len(X_train) > 0 and len(X_test) > 0:
                # train model and predict
                y_pred = train_predict_num(X_train, y_train, X_test)

                # impute
                df.loc[nan_mask, col] = y_pred

            df.drop(columns='is_na', inplace=True)
        return df

        def get_feature_names_out(self, input_feat = None):
            if input_feat is None:
                return np.array(self.feature_names_in_)
            else:
                return np.array(input_feat)

I created custom imputers for the numerical and categorical columns with model prediction. Random forest was used as the prediction model.

In the imputers, the fit function returns self because there is nothing the imputer is learning from.

In the transform function, the imputing process takes place. The data goes through preprocessing steps where it uses the data with no missing values as features and those with missing values as the target. The model is fit on the training data and is then used to predicted the missing values. The missing values are imputed back to the dataframe using the index of the missing values gotten at the start.

Each imputer wrapped in a class and inherits functions from the super classes TransformerMixin and BaseEstimator to make the function work as an imputer.

### Base Prediction

In [ ]:
# creating copies to avoid altering the main data during testing scenarios
X_train_1, X_test_1, y_train_1, y_test_1 = X_train.copy(), X_test.copy(), y_train.copy(), y_test.copy()

In [ ]:
# preprocessing
num_transformer = Pipeline(steps=[
    ('num_imputer', NumImputer()),
    ('scaler', StandardScaler())
])

cat_transformer = Pipeline(steps=[
    ('cat_imputer', CatImputer()),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
        ('num', num_transformer, num_cols),
        ('cat', cat_transformer, cat_cols),

    ])

In [ ]:
# create a function to predict and evaluate
def predict_evaluate(X_train, X_test, y_train, y_test, model):
    num_cols = X_train.select_dtypes(include=np.number).columns.tolist()
    cat_cols = X_train.select_dtypes(exclude=np.number).columns.tolist()

    # preprocessor
    preprocessor = ColumnTransformer(transformers=[
        ('num', num_transformer, num_cols),
        ('cat', cat_transformer, cat_cols),

    ])

    #pipeline
    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    #fit
    model_pipeline.fit(X_train, y_train)

    # predict
    train_pred = model_pipeline.predict(X_train)
    test_pred = model_pipeline.predict(X_test)

    #accuracy
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)

    print(f"Train Accuracy: {train_acc}, Test Accuracy: {test_acc}")
    print(classification_report(y_test, test_pred))

In [ ]:
# using logistic regression as a base model
model = LogisticRegression(random_state=20, max_iter=1000, solver='liblinear')
predict_evaluate(X_train_1, X_test_1, y_train_1, y_test_1, model)

Considering that the overall goal of the model is to be able to detect possible defaulters, the metric I will be focusing on is recall. Recall of the 0 class (bad flags) gives the performance of the model's ability to detect defaulters. Right now with a recall of 0.11 on the 0 class, the model can only detect 11% bad flags which is very poor.

The goal is to improve the recall score of the model. Using the baseline model (Logistic Regression), I will do this through:
- Handling class imbalance
- Feature selection

Then I will test the data on 5 or more models and choose the one with the best recall score of the 0 class (bad flags). When choosing, I will also consider the precision ensuring that there is a good balance between catching most of the bad loans and not incorrectly flagging too many good ones. Then I will perform futher tuning on the chosen model.

Due to the size of the dataset, however, the model has very little to learn from and performance might not be very high.

### Handle Class Imbalance

In [ ]:
# ADASYN
X_train_processed = preprocessor.fit_transform(X_train)
adasyn = ADASYN(random_state=20)

X_adn, y_adn = adasyn.fit_resample(X_train_processed, y_train)

In [ ]:
y_adn.value_counts()

In [ ]:
# SMOTE
X_train_processed = preprocessor.fit_transform(X_train)
smote = SMOTE(random_state=20)

X_smote, y_smote = smote.fit_resample(X_train_processed, y_train)

In [ ]:
y_smote.value_counts()

In [ ]:
def predict_evaluate_over(X_train_resampled, X_test, y_train_resampled, y_test, model):
    #preprocess
    X_test_processed = preprocessor.transform(X_test)

    #fit
    model.fit(X_train_resampled, y_train_resampled)

    # predict
    train_pred = model.predict(X_train_resampled)
    test_pred = model.predict(X_test_processed)

    #accuracy
    train_acc = accuracy_score(y_train_resampled, train_pred)
    test_acc = accuracy_score(y_test, test_pred)

    print(f"Train Accuracy: {train_acc}, Test Accuracy: {test_acc}")
    print(classification_report(y_test, test_pred))

In [ ]:
# ADASYN
model = LogisticRegression(random_state=20, max_iter=1000)
predict_evaluate_over(X_adn, X_test_1, y_adn, y_test_1, model)

In [ ]:
# SMOTE
model = LogisticRegression(random_state=20, max_iter=1000)
predict_evaluate_over(X_smote, X_test_1, y_smote, y_test_1, model)

ADASYN gave a better recall score and precison-recall trade off with a precision of 24% and a recall of 62%. (SMOTE: precision 25%, recall 57%)

#### **Rationale for Feature Selection After Handling Imbalance**

After applying ADASYN resampling to address class imbalance, the model with all features showed improved recall on the training data. However, I will proceed with systematic feature selection for several strategic reasons:

1. Overfitting Risk: With synthetic data on a relatively small dataset (4,368 samples), using all features could cause the model to learn artificial patterns from the oversampled data rather than genuine predictive signals.

2. Model Generalization: Feature selection helps ensure consistent performance on real-world loan applications that don’t contain synthetic data characteristics.

3. Business Requirements: Fewer, more interpretable features are critical for lending decisions, regulatory compliance, and stakeholder understanding.

4. Production Readiness: Removing redundant or irrelevant features creates a more stable model that’s less sensitive to variations and easier to maintain operationally.

Approach: Instead of optimizing purely for training recall, I will prioritise building a robust, generalizable model. This means accepting some trade-offs in initial recall in favor of long-term reliability and business practicality.

### Feature Selection
- VIF
- Mutual Information


#### Variance Inflation Factor (VIF)
Check multicolinearity of chosen columns using VIF

In [ ]:
# preprocessing
def mini_preprocess(X_train):
    num_cols = X_train.select_dtypes(include=np.number).columns.tolist()
    cat_cols = X_train.select_dtypes(exclude=np.number).columns.tolist()

    for col in X_train.select_dtypes(["category"]).columns:
        X_train[col] = X_train[col].astype(str)

    X_train[num_cols] = X_train[num_cols].fillna(0)
    X_train[cat_cols] = X_train[cat_cols].fillna("Missing")

    for col in cat_cols:
        codes, unique = pd.factorize(X_train[col])
        X_train[col] = codes

    X_train = X_train.fillna(0)

    return X_train

In [ ]:
X_train_1 = mini_preprocess(X_train_1)

In [ ]:
def calculate_vif(data):
    vif=pd.DataFrame()
    vif['features'] = data.columns
    vif['VIF_Value'] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]

    return vif

In [ ]:
calculate_vif(X_train_1)

In [ ]:
X_train_1_vred = X_train_1.drop(['age_is_missing', 'age_group_is_missing', 'bank_account_type_is_missing', 'bank_name_clients_is_missing',
                                'no_default_count', 'early_count', 'on_time_count', 'prev_loan_number', 'interest_amount', 'loannumber',
                                'prev_max_loan_amount', 'interest_rate', 'loanamount', 'approval_hour', 'approval_speed_hours'], axis=1)
cols_drop = ['age_is_missing', 'age_group_is_missing', 'bank_account_type_is_missing', 'bank_name_clients_is_missing', 'no_default_count',
             'early_count', 'on_time_count', 'prev_loan_number', 'interest_amount', 'loannumber', 'prev_max_loan_amount', 'interest_rate',
             'loanamount', 'approval_hour', 'approval_speed_hours']

In [ ]:
# After dropping columns and rechecking the vif value, the above columns were chosen to be dropped

In [ ]:
calculate_vif(X_train_1_vred)

In [ ]:
#create copies
X_train_1, X_test_1, y_train_1, y_test_1 = X_train.copy(), X_test.copy(), y_train.copy(), y_test.copy()

In [ ]:
X_train_1_red = X_train_1.drop(cols_drop, axis=1)
X_test_1_red = X_test_1.drop(cols_drop, axis=1)

In [ ]:
def predict_evaluate_adn(X_train, X_test, y_train, y_test, model):
    num_cols = X_train.select_dtypes(include=np.number).columns.tolist()
    cat_cols = X_train.select_dtypes(exclude=np.number).columns.tolist()

    # preprocessor
    preprocessor = ColumnTransformer(transformers=[
        ('num', num_transformer, num_cols),
        ('cat', cat_transformer, cat_cols),

    ])

    # preprocess
    X_train_preprocessed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)
    # resample
    X_train_adn, y_train_adn = adasyn.fit_resample(X_train_preprocessed, y_train)

    #fit
    model.fit(X_train_adn, y_train_adn)

    # predict
    train_pred = model.predict(X_train_adn)
    test_pred = model.predict(X_test_processed)

    #accuracy
    train_acc = accuracy_score(y_train_adn, train_pred)
    test_acc = accuracy_score(y_test, test_pred)

    print(f"Train Accuracy: {train_acc}, Test Accuracy: {test_acc}")
    print(classification_report(y_test, test_pred))

In [ ]:
model = LogisticRegression(random_state=20, max_iter=1000, solver='liblinear')
predict_evaluate_adn(X_train_1_red, X_test_1_red, y_train_1, y_test_1, model)

#### Mutual Information
Pick top features based on their relevance using mutual information

In [ ]:
X_train_1_red = mini_preprocess(X_train_1_red)
X_adn, y_adn = adasyn.fit_resample(X_train_1_red, y_train_1)

In [ ]:
mutual_info = mutual_info_classif(X_adn, y_adn, random_state=20)

In [ ]:
mutual_info = pd.Series(mutual_info).sort_values(ascending=False)
mutual_info.index = X_train_1_red.columns
mutual_info

In [ ]:
k = 10  # pick top 10
top_k_features = mutual_info.head(k).index
print(top_k_features)

In [ ]:
cols = ['termdays', 'bank_account_type', 'bank_name_clients', 'employment_status_clients', 'default_count', 'default_rate',
        'prev_total_loan', 'approval_day', 'loan_amount_category', 'loan_number_category']
X_train_mi = X_train_1.loc[:, cols]
X_test_mi = X_test_1.loc[:, cols]

In [ ]:
model = LogisticRegression(random_state=20, max_iter=1000, solver='liblinear')
predict_evaluate_adn(X_train_mi, X_test_mi, y_train, y_test, model)

#### **Feature Selection Approach:**
I tested two feature selection methods across different machine learning models to find the best approach for detecting loan defaults, with final selection validated against business insights to ensure practical relevance in loan default prediction.

1. First Test: Using only VIF to remove related features improved logistic regression recall from 11% to 50%. But when I tried this same feature set on other models like SVC and random forest, they performed poorly.
2. Second Test: Using mutual information + VIF gave logistic regression 42% recall at first - lower than the 50%. However, this feature set worked much better across different models, enabling fair comparison of performance.

**Why I Chose Mutual Information:**

Even though it reduced logistic regression performance, this method allowed me to test different models on equal footing and identify the best model for this problem. Testing multiple models with consistent features provided a more comprehensive approach than optimizing for a single model.

**Alternative Approach:**

I could have tuned the logistic regression with VIF features to try improving the 50% recall, but using mutual information features created a better foundation for systematic model comparison and selection.

Through feature selection using VIF and the mutual information, I have been able to improve the base model's recall from 11% to 42%



## Training, Predicting with and Evaluating Multiple Models

In [ ]:
models = {
    'Logistic Regression': LogisticRegression(solver='liblinear', max_iter=1000, random_state=20),
    'Decision Tree' : DecisionTreeClassifier(random_state=20),
    'Random Forest': RandomForestClassifier(random_state=20),
    'Gradient Boost' : GradientBoostingClassifier(random_state=20),
    'XG Boost' : XGBClassifier(random_state=20),
    'SVC' : SVC(random_state=20)
}

In [ ]:
# prediction function
def predict_evaluate_adn_models(X_train, X_test, y_train, y_test, model_name, model):
    num_cols = X_train.select_dtypes(include=np.number).columns.tolist()
    cat_cols = X_train.select_dtypes(exclude=np.number).columns.tolist()

    # preprocessor
    preprocessor = ColumnTransformer(transformers=[
        ('num', num_transformer, num_cols),
        ('cat', cat_transformer, cat_cols),

    ])

    # preprocess
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)
    # resample
    X_train_adn, y_train_adn = adasyn.fit_resample(X_train_processed, y_train)

    #fit
    model.fit(X_train_adn, y_train_adn)

    # predict
    train_pred = model.predict(X_train_adn)
    test_pred = model.predict(X_test_processed)

    #accuracy
    train_acc = accuracy_score(y_train_adn, train_pred)
    test_acc = accuracy_score(y_test, test_pred)

    # confusion matrix
    train_cm = confusion_matrix(y_train_adn, train_pred)
    test_cm = confusion_matrix(y_test, test_pred)

    return {
        'train_acc': train_acc,
        'test_acc': test_acc,
        'train_cm': train_cm,
        'test_cm': test_cm,
        'classification_report': classification_report(y_test, test_pred, output_dict=True)
    }


In [ ]:
# predict and evaluate models
results = {}

for name, model in models.items():
    results[name] = predict_evaluate_adn_models(X_train_mi, X_test_mi, y_train, y_test, name, model)

In [ ]:
fig, (axes1, axes2) = plt.subplots(2, 6, figsize=(20, 4), constrained_layout=True)

for (i, (name, result)), ax1, ax2 in zip(enumerate(results.items()), axes1, axes2):
    train_cm = result['train_cm']
    test_cm = result['test_cm']

    #plot confusion matrix
    disp_train = ConfusionMatrixDisplay(train_cm)
    disp_test = ConfusionMatrixDisplay(test_cm)

    #train cm
    disp_train.plot(ax=ax1, cmap='Blues')
    ax1.set_title(f'{name} Train Confusion Matrix')

    #test cm
    disp_test.plot(ax=ax2, cmap='Blues')
    ax2.set_title(f'{name} Test Confusion Matrix')

In [ ]:
results_df = pd.DataFrame(results).T
results_df

In [ ]:
report_dfs = {}

for model, report in results_df['classification_report'].items():
    df = pd.DataFrame(report).T
    report_dfs[model] = df

In [ ]:
all_reports = pd.concat(report_dfs, axis=0)
display(all_reports)

Based on the recall and precision trade off, the top model is SVC which has 35% precision and 41% recall on the 0 class (bad flags).

Train accuracy, Test accuracy = (0.720, 0.708)

## Hyperparameter Tuning

In [ ]:
model = SVC(random_state=20)

In [ ]:
# using grid search csv

#define parameters
parameters= {
    'C': [0.1, 10],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'class_weight': ['balanced', None]
}

In [ ]:
# create a function for tuning
def tune_model(X_train, X_test, y_train, y_test, model, parameters):
    num_cols = X_train.select_dtypes(include=np.number).columns.tolist()
    cat_cols = X_train.select_dtypes(exclude=np.number).columns.tolist()

    # preprocessor
    preprocessor = ColumnTransformer(transformers=[
        ('num', num_transformer, num_cols),
        ('cat', cat_transformer, cat_cols),

    ])

    # preprocess
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)

    # search
    search = GridSearchCV(estimator=model, param_grid=parameters, n_jobs=-1, cv=5, scoring=make_scorer(recall_score, pos_label=0))
    search.fit(X_train_processed, y_train)

    best_model = search.best_estimator_
    return best_model

In [ ]:
#tune_model(X_train_mi, X_test_mi, y_train, y_test, model, parameters)

In [ ]:
final_model = SVC(C=0.1, class_weight='balanced', gamma=1, kernel='sigmoid', random_state=20, probability=True)

In [ ]:
# train, predict and evaluate

num_cols = X_train_mi.select_dtypes(include=np.number).columns.tolist()
cat_cols = X_train_mi.select_dtypes(exclude=np.number).columns.tolist()

# preprocessor
preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_cols),
    ('cat', cat_transformer, cat_cols),

])

# preprocess
X_train_mi_processed = preprocessor.fit_transform(X_train_mi)
X_test_mi_processed = preprocessor.transform(X_test_mi)
# resample
X_train_adn, y_train_adn = adasyn.fit_resample(X_train_mi_processed, y_train)

#fit
final_model.fit(X_train_adn, y_train_adn)

# predict
train_pred = final_model.predict(X_train_adn)
test_pred = final_model.predict(X_test_mi_processed)

#accuracy
train_acc = accuracy_score(y_train_adn, train_pred)
test_acc = accuracy_score(y_test, test_pred)

print(f"Train Accuracy: {train_acc}, Test Accuracy: {test_acc}")
print(classification_report(y_test, test_pred))

## SHAP

In [ ]:
# get train and test samples
background_info = X_train_mi_processed[:50]
test_sample = X_test_mi_processed[:20]

# get feature names for interpretability
feature_names = (num_cols +
                list(preprocessor.named_transformers_['cat'].named_steps['encoder'].get_feature_names_out(cat_cols)))

# get shap values
explainer = shap.KernelExplainer(final_model.predict_proba, background_info)
shap_values = explainer.shap_values(test_sample)

# check shape to ensure compatibility
print(background_info.shape, test_sample.shape, len(feature_names), shap_values[0].shape)

In [ ]:
# correct shap values shape
shap_values_corrected = [np.array([shap_values[i][:, 0] for i in range(20)]), np.array([shap_values[i][:, 1] for i in range(20)])]

In [ ]:
# summary bar plot
shap.summary_plot(shap_values_corrected[0], test_sample, feature_names=feature_names, plot_type="bar")

In [ ]:
#drop the least contributing feature - approval day
X_train_shap = X_train_mi.drop('approval_day', axis=1)
X_test_shap = X_test_mi.drop('approval_day', axis=1)

In [ ]:
# train, predict and evaluate

num_cols = X_train_shap.select_dtypes(include=np.number).columns.tolist()
cat_cols = X_train_shap.select_dtypes(exclude=np.number).columns.tolist()

# preprocessor
preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_cols),
    ('cat', cat_transformer, cat_cols),

])

# preprocess
X_train_shap_processed = preprocessor.fit_transform(X_train_shap)
X_test_shap_processed = preprocessor.transform(X_test_shap)
# resample
X_train_adn, y_train_adn = adasyn.fit_resample(X_train_shap_processed, y_train)

#fit
final_model.fit(X_train_adn, y_train_adn)

# predict
train_pred = final_model.predict(X_train_adn)
test_pred = final_model.predict(X_test_shap_processed)

#accuracy
train_acc = accuracy_score(y_train_adn, train_pred)
test_acc = accuracy_score(y_test, test_pred)

print(f"Train Accuracy: {train_acc}, Test Accuracy: {test_acc}")
print(classification_report(y_test, test_pred))

In [ ]:
# confusion matrix
test_cm = confusion_matrix(y_test, test_pred)
disp_test = ConfusionMatrixDisplay(test_cm)
disp_test.plot(cmap='Blues')
plt.show()

The confusion matrix shows the model correctly identified 43 bad loans and 158 good loans. However, it misclassified 33 bad borrowers as good and 115 good borrowers as bad.

### **Predictive Model Analysis - Conclusion**

#### **Model Development and Performance**

I developed an SVC (Support Vector Classifier) to predict loan defaults using a feature selection approach that combined VIF and mutual information analysis with business insights from previous loan analysis. This ensured the model captures both statistical significance and real-world lending risk patterns.

**Final Model Performance:**
- Precision for Bad Loans: 27%
- Recall for Bad Loans: 57%
- F1-Score for Bad Loans: 37%
- Overall Test Accuracy: 58%

The model successfully identifies 57% of actual defaults, meaning it catches more than half of risky loans before they occurred. While precision is low at 27%, this trade off is acceptable in lending situations where missing a default (false negative) is typically more costly than declining a good applicant (false positive).

#### **Feature Selection and Model Refinement**

I used a hybrid approach combining statistical methods (VIF and mutual information) with business insights to select features. SHAP analysis further validated this approach by identifying `approval_day` as the least contributing feature. Removing this feature improved recall from 55% to 57% while maintaining precision, resulting in a final model with 9 highly relevant features focused on borrower behavior, demographics, and loan characteristics.

#### **Dataset Limitations and Model Constraints**

With 4,368 rows and a 78% vs 22% class imbalance, the dataset presents several challenges:
- Limited Sample Size: Insufficient for highly complex models or deep feature engineering
- Class Imbalance: I addressed this through ADASYN resampling, balanced class weights, and recall-focused optimization
- Generalization Concerns: Model performance may vary with new data due to sample size constraints

#### **Business Impact and Recommendations**
**Immediate Value:**
- Risk Reduction: Successfully identifying 57% of defaults represents significant potential savings for the lender.
- Automated Screening: The model can flag high-risk applications for manual review
- Decision Support: Insights support informed lending decisions

**Implementation Recommendations:**
- Pilot Deployment: Test the model on new applications under human supervision.
- Threshold Adjustment: Adjust prediction thresholds to capture more defaults if the business can tolerate a higher false positive rate.
- Continuous Monitoring: Track model performance on new data and retrain as needed
- Data Expansion: Collect additional borrower and loan data to improve model robustness and accuracy.

#### **Future Enhancements**
1. Larger Dataset: Expand to 10,000+ samples for improved statistical reliability
2. Advanced Models: Explore ensemble methods combining SVC with tree-based models or or other state-of-the-art algorithms.
3. Feature Engineering: Develop richer behavioral and temporal features for better predictive power.
4. Real-time Integration: Implement systems for live predictions and real-time decision support.

#### **Final Assessment**

While constrained by dataset size, I successfully built a loan default prediction model that achieves meaningful business value. The 57% recall rate for default identification, combined with SHAP-validated feature selection, provides a strong foundation for operational implementation and future model enhancement.